Thusfar, the **sourceCpp** function from the **Rcpp** package has been used to process c++ written into a string variable within an **R** session. This is useful, but can soon become inconvenient. Alternatively, th **c++** code can be saved in a file and the name of this file provided as an argument to the **sourceCpp** function. For instance, the example from sction 1 could be saved into a file **example_1.cpp**, in which case 

In [36]:
sourceCpp(file="example_1.cpp")


Warning message in normalizePath(file, winslash = "/"):
“path[1]="example_1.cpp": No such file or directory”


ERROR: Error in eval(expr, envir, enclos): file not found: 'example_1.cpp'


This facility is helpful. However, most **c++*programs* programs will be organised into more than one file. Unfortunately, the **sourceCpp** function has nothing to offer to help with this, and can only "manage" a single source file.

## A minimal Rcpp package

**R** provides a mechanism for grouping related files containing, amongst other things, **R** code, **c++** code, documentation, data files, vignettes, and so on. This mechanism also supports the management of package dependencies, version control, package citations and so on.

Although the R packaging mechanism is distinct from **Rcpp**, it can be used in conjunction with it. Afterall, **Rcpp** is an **R** package itself. One of the advantages of doing this (from the **Rcpp** perspective) is that the **R** package mechanism enables *8Rcpp** to work with multiple **c++* source files.

Ultimately, an **R** package is just a collection of appropriately named files in a prescribed directory structure. There are many tools to help you construct an **R** package, but the minimal **Rcpp** enabled package is quite simple to create. The figure below shows the basic directory structure and files needed to create a minimal **Rcpp** package.

<img src="./images/package-structure.png">

Here, **praxi** is the name of the package being created.

### Exercise 

Create the above directory structure.

### The DESCRIPTION file

The **DESCRIPTION** file contains a basic description of the package (named **praxi** in this case) and other information such as the package maintainer detalis and other R packages that **praxi** is dependent on. Notice  that that the Imports: and LinkingTo: fileds in the **DESCRIPTION** file list the **Rcpp** package.

```R
Package: praxi
Type: Package
Title: What the Package Does in One 'Title Case' Line
Version: 1.0
Date: 2024-02-29
Author: Your Name
Maintainer: Your Name <your@email.com>
Description: One paragraph description of what the package does as one
        or more full sentences.
License: GPL (>= 2)
Imports: Rcpp (>= 1.0.12)
LinkingTo: Rcpp
```

### Exercise 

Create the **DESCRIPTION** file and place it into the **praxi** package directory structure. Note, you can set the name of the package using the 

```R
Package: 
```

field of the **DESCRIPTION** file.

### The NAMESPACE file

The **NAMESPACE** file also lists the dependencis of the package. It also registers the library when it is installed and exports the methods in the package for a user to access when the package is loaded using the **library** function within an **R** session or script. Notice that the package name appears in the **useDynLib** directive.


```R
useDynLib(praxi, .registration=TRUE)
import(methods, Rcpp)
exportPattern("^[[:alpha:]]+")
```

### Exercise 

Create the **NAMESPACE** file and place it into the **praxi** package directory structure.

### The load_Rcpp_module.R file¶

This file is used to load **Rcpp** modules into an R session. 

```c++
loadModule("marshalling", TRUE)
```

Note, that the file does **not** have to be called load_Rcpp_module.R, but this is a good descriptive choice. It can also contain multiple loadModule directives (if you have more than one module to load. Of course, the module names correspond to those you use in your augmented **c++** code.

### Exercise 

Create the **load_Rcpp_module.R** file and place it into the **praxi** package directory structure.

### The Makevars file

The **Makevars** file is optional. It can be used to configure the way in which the R package gets constructed. Importantly, it can contain information about which version of **c++** compiler to use when creating the package.

```R
CXX_STD=CXX17
```

In this case, a **c++ 17** compiler is specified.

### Exercise 

Create the **Makevars** file and place it into the **praxi** package directory structure.

### The c++ source and header files

To add **c++** code to the package, place your source code (*.cpp) and header (*.h) files in the **src** directory. This source code should include any **Rcpp** specific code, such as the module definitions. For example

```c++
#include "Rcpp.h"
using namespace Rcpp;


#include <string>
#include <iostream>


std::string marshall_string(const std::string& X)
{
    Rcout << X << std::endl;
    std::string Y {" blady blah ..."};
    return X + Y;
}




RCPP_MODULE(marshalling) 
{
function("rcpp_marshall_string", &marshall_string);
}
```

Note that you can seperate your source code from the module definitions by using header files. Each *.cpp file in the src directory will be compiled and made available for use by other **c++** functions in your package. However, only those functions added to the module will be avaialble to users of your package.

## Building and installing an R pakage